In [1]:
!pip install transformers

In [1]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
  logp = F. log_softmax(logits, dim=- 1)
  logp_label = torch.gather(logp, 2, labels. unsqueeze(2)). squeeze(- 1)
  return logp_label

In [2]:
def sequence_logprob(model, labels, input_len=0):
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(output.logits[:, : - 1, :], labels[:, 1:])
    seq_log_prob = torch.sum(log_probs[:, input_len:])
  return seq_log_prob.cpu().numpy()


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch. cuda. is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English. \n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt" )["input_ids" ].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)

logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f" \nlog-prob: {logp:.2f}" )


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees. 


The researchers were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees. 


The researchers were conducting a study on
 
log-prob: -68.37


In [4]:
output_beam = model.generate(input_ids,
                              max_length=max_length,
                              num_beams=5,
                              do_sample=False)

logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))

print(tokenizer. decode(output_beam[0]))
print(f" \nlog-prob: {logp:.2f}" )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


According to the researchers, the unicorns were found in a remote valley in the Andes Mountains in Peru. The valley is located in a remote area of the Andes Mountains. The valley is located in a remote area of the Andes Mountains. According to the researchers, the unicorns were found in a remote valley in the Andes Mountains in Peru. The valley is located in a remote area
 
log-prob: -44.86


In [5]:
output_beam = model.generate(input_ids,
                              max_length=max_length,
                              num_beams=5,
                              do_sample=False,
                              no_repeat_ngram_size=2)

logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))

print(tokenizer.decode(output_beam[0]))
print(f" \nlog-prob: {logp:.2f}" )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 


The researchers, from the University of California, Los Angeles (UCLA) and the Universidad Nacional Autónoma de México (UNAM) in Mexico City, discovered the unicorn herd by accident. They were conducting a study on the effects of climate change on wild animals, when they came across the herd.

"When we first saw them, we couldn't believe our
 
log-prob: -77.14
